In [17]:
# -----------------------------
# 1️⃣ Imports
# -----------------------------
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from imblearn.combine import SMOTETomek  # The best-performing sampler
from sklearn.metrics import classification_report, confusion_matrix
import joblib

# -----------------------------
# 2️⃣ Load Dataset & Feature Engineering
# -----------------------------
df = pd.read_excel("P585 Churn.xlsx")

df.columns = df.columns.str.strip()
df.replace(['Nan', 'nan', 'NaN', 'NAN'], np.nan, inplace=True)
df.drop(columns=['Unnamed: 0'], errors='ignore', inplace=True)

# Create 'value' features
df['total_charge'] = df['day.charge'] + df['eve.charge'] + df['night.charge'] + df['intl.charge']
df['total_mins'] = df['day.mins'] + df['eve.mins'] + df['night.mins'] + df['intl.mins']
df['charge_per_min'] = df['total_charge'] / (df['total_mins'] + 1e-6)

# Create 'usage' features
df['total_calls'] = df['day.calls'] + df['eve.calls'] + df['night.calls'] + df['intl.calls']
df['mins_per_call'] = df['total_mins'] / (df['total_calls'] + 1e-6)

# Bin the 'customer.calls' feature
def get_service_call_bin(calls):
    if calls == 0:
        return '0_calls'
    elif calls <= 3:
        return '1-3_calls'
    else:
        return '4+_calls'

df['service_call_bin'] = df['customer.calls'].apply(get_service_call_bin)

# Create the "Pay-As-You-Go International" pain feature
df['pay_as_you_go_intl'] = (
    (df['intl.plan'] == 'no') & (df['intl.mins'] > 0)
).astype(int)

# -----------------------------
# 3️⃣ Feature & Target
# -----------------------------
X = df.drop('churn', axis=1)
y = df['churn']

# -----------------------------
# 4️⃣ Train-Test Split
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# -----------------------------
# 5️⃣ Impute + Encode
# -----------------------------
categorical_cols = ['state', 'voice.plan', 'intl.plan', 'service_call_bin']
numeric_cols = X_train.select_dtypes(include=np.number).columns

num_imputer = SimpleImputer(strategy='median')
cat_imputer = SimpleImputer(strategy='most_frequent')

X_train_num = num_imputer.fit_transform(X_train[numeric_cols])
X_test_num = num_imputer.transform(X_test[numeric_cols])

X_train_cat = cat_imputer.fit_transform(X_train[categorical_cols])
X_test_cat = cat_imputer.transform(X_test[categorical_cols])

ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_train_enc = ohe.fit_transform(X_train_cat)
X_test_enc = ohe.transform(X_test_cat)

X_train_final = np.hstack([X_train_num, X_train_enc])
X_test_final = np.hstack([X_test_num, X_test_enc])

# -----------------------------
# 6️⃣ Scaling
# -----------------------------
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_final)
X_test_scaled = scaler.transform(X_test_final)

# -----------------------------
# 7️⃣ Resampling with SMOTETomek
# -----------------------------
smote_tomek = SMOTETomek(random_state=42)
X_res, y_res = smote_tomek.fit_resample(X_train_scaled, y_train)

# -----------------------------
# 8️⃣ XGBoost Model + RandomizedSearch
# -----------------------------
y_res_encoded = (y_res == 'yes').astype(int)
y_test_encoded = (y_test == 'yes').astype(int)

# Define the parameter grid to search
param_grid_xgb = {
    'n_estimators': [200, 300],
    'max_depth': [6, 7, 8],
    'learning_rate': [0.1, 0.05],
    'subsample': [0.7, 1.0],
    'colsample_bytree': [0.7, 1.0],
    'gamma': [0, 0.1, 0.2],
    'reg_alpha': [0, 0.1],
    'reg_lambda': [1, 1.5]
}

# Use RandomizedSearchCV
grid_search_xgb = RandomizedSearchCV(
    estimator=XGBClassifier(
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42
    ),
    param_distributions=param_grid_xgb,
    n_iter=50,  # Try 50 random combinations
    scoring='f1',
    cv=5,
    n_jobs=-1,
    verbose=1,
    random_state=42
)

# Fit on the resampled data
grid_search_xgb.fit(X_res, y_res_encoded)

print("Best XGBoost Params:", grid_search_xgb.best_params_)

# Use the BEST model found by the search
xgb_model = grid_search_xgb.best_estimator_

# -----------------------------
# 9️⃣ Threshold Prediction Function
# -----------------------------
def predict_with_threshold(model, X, threshold=0.5):
    probs = model.predict_proba(X)[:, 1]
    pred = np.where(probs >= threshold, 'yes', 'no')
    return pred

# -----------------------------
# 🔟 Final Evaluation on XGBoost
# -----------------------------
y_pred = predict_with_threshold(xgb_model, X_test_scaled, threshold=0.5)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# -----------------------------
# 1️⃣1️⃣ Save Artifacts for Deployment
# -----------------------------
joblib.dump(num_imputer, "num_imputer.pkl")
joblib.dump(cat_imputer, "cat_imputer.pkl")
joblib.dump(ohe, "encoder.pkl")
joblib.dump(scaler, "scaler.pkl")
joblib.dump(xgb_model, "xgb_model.pkl") # Save the best model

print("✅ Saved: num_imputer.pkl, cat_imputer.pkl, encoder.pkl, scaler.pkl, xgb_model.pkl")

/tmp/ipython-input-1971731295.py:20: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace(['Nan', 'nan', 'NaN', 'NAN'], np.nan, inplace=True)


Fitting 5 folds for each of 50 candidates, totalling 250 fits


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [11:38:40] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Best XGBoost Params: {'subsample': 1.0, 'reg_lambda': 1, 'reg_alpha': 0, 'n_estimators': 300, 'max_depth': 8, 'learning_rate': 0.05, 'gamma': 0.2, 'colsample_bytree': 0.7}
              precision    recall  f1-score   support

          no       0.98      1.00      0.99       859
         yes       0.98      0.87      0.92       141

    accuracy                           0.98      1000
   macro avg       0.98      0.93      0.95      1000
weighted avg       0.98      0.98      0.98      1000

[[856   3]
 [ 19 122]]
✅ Saved: num_imputer.pkl, cat_imputer.pkl, encoder.pkl, scaler.pkl, xgb_model.pkl
